# Projet maison n° 3

In [9]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [10]:
# names us
def df_names_us():
    df = pd.DataFrame()

    #lecture des fichiers 1 par 1 et ajout au DataFrame global
    for k in range(1880,2022) :
        file = 'yob' + str(k) + '.txt'
        current_df = pd.read_csv(file, names = ['name', 'gender', 'births'],delimiter=",")
        current_df['year'] = k
        df = pd.concat([df,current_df],ignore_index=True)
    df = df[['year','name','gender','births']]
    return df
print(df_names_us())


         year       name gender  births
0        1880       Mary      F    7065
1        1880       Anna      F    2604
2        1880       Emma      F    2003
3        1880  Elizabeth      F    1939
4        1880     Minnie      F    1746
...       ...        ...    ...     ...
2052776  2021     Zyeire      M       5
2052777  2021       Zyel      M       5
2052778  2021      Zyian      M       5
2052779  2021      Zylar      M       5
2052780  2021        Zyn      M       5

[2052781 rows x 4 columns]


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [11]:
# names fr
def df_names_fr():
    

    #lecture fichier csv
    df = pd.read_csv('nat2021.csv', names = ['gender', 'name', 'year', 'births' ], delimiter= ';', dtype= str).drop(index = 0) 
    
    #suppression des données inutilisables et reformatage des prénoms
    df = df[df['name'] != '_PRENOMS_RARES'] 
    df = df[df['name'].apply(lambda x : len(str(x))) != 1]  
    df['name'] = df['name'].apply(lambda x : str(x).title()) 
    df = df[df['year'] != 'XXXX'] 
    
    #mapping du genre
    mapping = pd.Series({'2': 'F', 2 : 'F', '1': 'M', 1 : 'M'}) 
    df['gender'] = df['gender'].map(mapping)
    
    #reformatage et tri des colonnes
    df = df[['year','name','gender','births']]
    df['births'] = df['births'].astype(int)
    df = df.sort_values(by = ['year' , 'gender', 'births', 'name'] , ascending= [True, True, False, True]) #tri selon les différents critères
    

    df.index = pd.RangeIndex(len(df))
    

    return df


print(df_names_fr())

        year        name gender  births
0       1900       Marie      F   48713
1       1900      Jeanne      F   13981
2       1900  Marguerite      F    8058
3       1900    Germaine      F    6981
4       1900      Louise      F    6696
...      ...         ...    ...     ...
648325  2021       Élyam      M       3
648326  2021       Énael      M       3
648327  2021        Éros      M       3
648328  2021        Étan      M       3
648329  2021    Évariste      M       3

[648330 rows x 4 columns]


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [12]:
# taux de change
from cmath import nan
from dataclasses import replace


def df_taux_change(devises):
    df = pd.read_csv('Webstat_Export_20220924.csv', delimiter= ';', header = 0, dtype = str).drop(index=[0,1,2,3,4])
    
    # transformation des noms de colonnes avec le code ISO3
    columns = df.columns
    columns = map(lambda x : x[x.find("(")+1:x.find(")")], columns) 
    df.columns = columns
    df = df[['Titre '] + devises]

    # construction de l'index du dataframe sur les dates
    df = df.rename(columns = {'Titre ' : 'Date'})
    df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y")
    df = df.sort_values(by= ['Date'], ascending=[True])
    df = df.set_index('Date')
    
    # suppression des colonnes dupliquées
    duplicates = df.columns.duplicated()
    df = df.loc[:, ~duplicates]

    # conversion en flottants et supression des NaN
    df = df.dropna()
    df = df.applymap(lambda x : x.replace(',' , '.'))
    df = df.applymap(lambda x : x.replace('-' , 'NaN'))
    df = df.applymap(lambda x : float(x))
    df = df.dropna()

    return df




df = df_taux_change(['CHF', 'GBP', 'USD'])
print(df)



               CHF      GBP     USD
Date                               
1999-01-04  1.6168  0.71110  1.1789
1999-01-05  1.6123  0.71220  1.1790
1999-01-06  1.6116  0.70760  1.1743
1999-01-07  1.6165  0.70585  1.1632
1999-01-08  1.6138  0.70940  1.1659
...            ...      ...     ...
2022-09-19  0.9658  0.87785  0.9990
2022-09-20  0.9644  0.87395  0.9986
2022-09-21  0.9549  0.87335  0.9906
2022-09-22  0.9684  0.87256  0.9884
2022-09-23  0.9565  0.88201  0.9754

[6078 rows x 3 columns]


### Tests

In [13]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [14]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [15]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 9.178s

OK
